# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 2 2022 8:30AM,245566,10,SOTAH0000769,"Nextsource Biotechnology, LLC",Released
1,Sep 1 2022 3:48PM,245565,19,8472301,"Avet Pharmaceuticals Labs, Inc",Released
2,Sep 1 2022 3:47PM,245564,16,SEQIRUS-8472296,Seqirus,Released
3,Sep 1 2022 3:40PM,245563,16,SEQIRUS-8472293,Seqirus,Released
4,Sep 1 2022 3:38PM,245562,20,8457816,"ACI Healthcare USA, Inc.",Released
5,Sep 1 2022 3:38PM,245562,20,8457902,"ACI Healthcare USA, Inc.",Released
6,Sep 1 2022 3:38PM,245562,20,8458026,"ACI Healthcare USA, Inc.",Released
7,Sep 1 2022 3:38PM,245562,20,8466162,"ACI Healthcare USA, Inc.",Released
8,Sep 1 2022 3:38PM,245562,20,8466163,"ACI Healthcare USA, Inc.",Released
9,Sep 1 2022 3:38PM,245562,20,8466719,"ACI Healthcare USA, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
22,245562,Released,14
23,245563,Released,1
24,245564,Released,1
25,245565,Released,1
26,245566,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
245562,NaN,NaN,14.0
245563,NaN,NaN,1.0
245564,NaN,NaN,1.0
245565,NaN,NaN,1.0
245566,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
245453,1.0,0.0,9.0
245455,0.0,3.0,3.0
245478,10.0,16.0,15.0
245500,0.0,0.0,1.0
245525,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
245453,1,0,9
245455,0,3,3
245478,10,16,15
245500,0,0,1
245525,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,245453,1,0,9
1,245455,0,3,3
2,245478,10,16,15
3,245500,0,0,1
4,245525,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,245453,1,,9
1,245455,,3,3
2,245478,10,16,15
3,245500,,,1
4,245525,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 2 2022 8:30AM,245566,10,"Nextsource Biotechnology, LLC"
1,Sep 1 2022 3:48PM,245565,19,"Avet Pharmaceuticals Labs, Inc"
2,Sep 1 2022 3:47PM,245564,16,Seqirus
3,Sep 1 2022 3:40PM,245563,16,Seqirus
4,Sep 1 2022 3:38PM,245562,20,"ACI Healthcare USA, Inc."
18,Sep 1 2022 3:12PM,245559,10,ISDIN Corporation
26,Sep 1 2022 3:06PM,245555,102,"Senseonics, Incorporated"
27,Sep 1 2022 3:06PM,245554,20,Alumier Labs Inc.
28,Sep 1 2022 3:04PM,245528,20,Alumier Labs Inc.
40,Sep 1 2022 3:03PM,245455,20,Alumier Labs Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 2 2022 8:30AM,245566,10,"Nextsource Biotechnology, LLC",,,1
1,Sep 1 2022 3:48PM,245565,19,"Avet Pharmaceuticals Labs, Inc",,,1
2,Sep 1 2022 3:47PM,245564,16,Seqirus,,,1
3,Sep 1 2022 3:40PM,245563,16,Seqirus,,,1
4,Sep 1 2022 3:38PM,245562,20,"ACI Healthcare USA, Inc.",,,14
5,Sep 1 2022 3:12PM,245559,10,ISDIN Corporation,,,8
6,Sep 1 2022 3:06PM,245555,102,"Senseonics, Incorporated",,,1
7,Sep 1 2022 3:06PM,245554,20,Alumier Labs Inc.,,,1
8,Sep 1 2022 3:04PM,245528,20,Alumier Labs Inc.,,4,8
9,Sep 1 2022 3:03PM,245455,20,Alumier Labs Inc.,,3,3


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 2 2022 8:30AM,245566,10,"Nextsource Biotechnology, LLC",1,,
1,Sep 1 2022 3:48PM,245565,19,"Avet Pharmaceuticals Labs, Inc",1,,
2,Sep 1 2022 3:47PM,245564,16,Seqirus,1,,
3,Sep 1 2022 3:40PM,245563,16,Seqirus,1,,
4,Sep 1 2022 3:38PM,245562,20,"ACI Healthcare USA, Inc.",14,,
5,Sep 1 2022 3:12PM,245559,10,ISDIN Corporation,8,,
6,Sep 1 2022 3:06PM,245555,102,"Senseonics, Incorporated",1,,
7,Sep 1 2022 3:06PM,245554,20,Alumier Labs Inc.,1,,
8,Sep 1 2022 3:04PM,245528,20,Alumier Labs Inc.,8,4,
9,Sep 1 2022 3:03PM,245455,20,Alumier Labs Inc.,3,3,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 2 2022 8:30AM,245566,10,"Nextsource Biotechnology, LLC",1,,
1,Sep 1 2022 3:48PM,245565,19,"Avet Pharmaceuticals Labs, Inc",1,,
2,Sep 1 2022 3:47PM,245564,16,Seqirus,1,,
3,Sep 1 2022 3:40PM,245563,16,Seqirus,1,,
4,Sep 1 2022 3:38PM,245562,20,"ACI Healthcare USA, Inc.",14,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 2 2022 8:30AM,245566,10,"Nextsource Biotechnology, LLC",1.0,NaN,NaN
1,Sep 1 2022 3:48PM,245565,19,"Avet Pharmaceuticals Labs, Inc",1.0,NaN,NaN
2,Sep 1 2022 3:47PM,245564,16,Seqirus,1.0,NaN,NaN
3,Sep 1 2022 3:40PM,245563,16,Seqirus,1.0,NaN,NaN
4,Sep 1 2022 3:38PM,245562,20,"ACI Healthcare USA, Inc.",14.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 2 2022 8:30AM,245566,10,"Nextsource Biotechnology, LLC",1.0,0.0,0.0
1,Sep 1 2022 3:48PM,245565,19,"Avet Pharmaceuticals Labs, Inc",1.0,0.0,0.0
2,Sep 1 2022 3:47PM,245564,16,Seqirus,1.0,0.0,0.0
3,Sep 1 2022 3:40PM,245563,16,Seqirus,1.0,0.0,0.0
4,Sep 1 2022 3:38PM,245562,20,"ACI Healthcare USA, Inc.",14.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2209830,76.0,5.0,0.0
102,245555,1.0,0.0,0.0
15,245478,15.0,16.0,10.0
16,982221,3.0,1.0,0.0
19,245565,1.0,0.0,0.0
20,1227552,35.0,7.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2209830,76.0,5.0,0.0
1,102,245555,1.0,0.0,0.0
2,15,245478,15.0,16.0,10.0
3,16,982221,3.0,1.0,0.0
4,19,245565,1.0,0.0,0.0
5,20,1227552,35.0,7.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,76.0,5.0,0.0
1,102,1.0,0.0,0.0
2,15,15.0,16.0,10.0
3,16,3.0,1.0,0.0
4,19,1.0,0.0,0.0
5,20,35.0,7.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,76.0
1,102,Released,1.0
2,15,Released,15.0
3,16,Released,3.0
4,19,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,15,16,19,20
Status,,,,,,
Completed,0.0,0.0,10.0,0.0,0.0,1.0
Executing,5.0,0.0,16.0,1.0,0.0,7.0
Released,76.0,1.0,15.0,3.0,1.0,35.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,15,16,19,20
0,Completed,0.0,0.0,10.0,0.0,0.0,1.0
1,Executing,5.0,0.0,16.0,1.0,0.0,7.0
2,Released,76.0,1.0,15.0,3.0,1.0,35.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,15,16,19,20
0,Completed,0.0,0.0,10.0,0.0,0.0,1.0
1,Executing,5.0,0.0,16.0,1.0,0.0,7.0
2,Released,76.0,1.0,15.0,3.0,1.0,35.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()